In [2]:
import time

import numpy as np
import tensorflow as tf

# Grouping Operations

## Transfering parameters from one set of variables to another

Example: Deep Q-Network, need to send "online" parameters to the "target" parameters periodically.

In [22]:
# Helper to create dummy graph with many value to be transferred from one Variable to another.
def create_var_updates_and_init():
    tf.reset_default_graph()
    print('Creating variables...')
    # Create dummy lists of variables for example
    master_vars = [tf.Variable(tf.random_normal([100, 100])) for i in range(100)]
    replica_vars = [tf.Variable(tf.random_normal([100, 100])) for i in range(100)]
    # Create assign ops for each variable
    update_ops = []
    for i, var in enumerate(replica_vars):
        master_var = master_vars[i]
        update_ops.append(var.assign(master_var))
    # initalization operation
    init = tf.global_variables_initializer()
    print('Done.')
    return update_ops, init

How should we run all of the `assign` ops?

I've seen things like this:

In [23]:
update_ops, init = create_var_updates_and_init()
# Run each update in a separate Session.run() call
with tf.Session() as sess:
    sess.run(init)
    start_t = time.time()
    for update in update_ops:
        sess.run(update)
    end_t = time.time()
    print('Time elapsed: {} seconds'.format(end_t - start_t))

Creating variables...
Done.
Time elapsed: 0.5819320678710938 seconds


We can do better.

In [24]:
update_ops, init = create_var_updates_and_init()
# Create one "master" operation which forces all update ops to execute
with tf.control_dependencies(update_ops):
    assign_all = tf.no_op()
with tf.Session() as sess:
    sess.run(init)
    start_t = time.time()
    sess.run(assign_all)
    end_t = time.time()
    print('Time elapsed: {} seconds'.format(end_t - start_t))

Creating variables...
Done.
Time elapsed: 0.014588594436645508 seconds


The syntax can be made even cleaner by using the `tf.group` operation:

In [25]:
update_ops, init = create_var_updates_and_init()
# Create one "master" operation which forces all update ops to execute
assign_all = tf.group(*update_ops)
with tf.Session() as sess:
    sess.run(init)
    start_t = time.time()
    sess.run(assign_all)
    end_t = time.time()
    print('Time elapsed: {} seconds'.format(end_t - start_t))

Creating variables...
Done.
Time elapsed: 0.015064001083374023 seconds


Using a grouping operation is cleaner, faster, more idiomatic. Compared to the `tf.control_dependencies()` version above, `tf.group()` provides some additional functionality under-the-hood, making sure that operations are grouped according to their device. [Check out the implementation here](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/ops/control_flow_ops.py#L2784-L2846).

## `tf.tuple`

If you want to synchronize multiple parallel operations, [`tf.tuple()`](https://www.tensorflow.org/versions/master/api_docs/python/tf/tuple) is an easy solution. You simply pass in a list of tensors, and `tf.tuple()` prevents future operations from using those tensors until they have all been computed.

_Note: TensorFlow operations automatically wait for all dependencies to finish before executing. Use `tf.tuple` for situations where you need to explicitly provide synchronization._

In [4]:
def make_matmul(dims):
    return tf.matmul(tf.random_normal([dims, dims]), tf.random_normal([dims, dims]))

In [3]:
tensor_a = make_matmul(100)
tensor_b = make_matmul(1000)
tensor_c = make_matmul(10000)
sync_a, sync_b, sync_c = tf.tuple([tensor_a, tensor_b, tensor_c])